In [ ]:
!pip install pandas
!pip install boto3
!pip install sagemaker
!pip install scikit-learn

In [ ]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
import numpy as np
import boto3
import pandas as pd
from sagemaker import get_execution_role
import sagemaker.amazon.common as smac
import os
import io

role = get_execution_role()
bucket='test-bucket-redha'
data_key = 'processed/processed_titanic.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

df = pd.read_csv(data_location)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,0
1,1,1,0,38.0,1,0,71.2833,2
2,1,3,0,26.0,0,0,7.9250,0
3,1,1,0,35.0,1,0,53.1000,0
4,0,3,1,35.0,0,0,8.0500,0


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
import numpy as np

from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(X_train, y_train, 'train.libsvm')
dump_svmlight_file(X_test, y_test, 'test.libsvm')

bucket_name = 'test-bucket-redha'
train_file = 'train.libsvm'
test_file = 'test.libsvm'

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join('train', train_file)).upload_file(train_file)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join('test', test_file)).upload_file(test_file)


container = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')

hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic",
    "num_round":50
}

estimator = sagemaker.estimator.Estimator(image_uri=container,
                                          hyperparameters=hyperparameters,
                                          role=role,
                                          train_instance_count=1,
                                          train_instance_type='ml.m5.xlarge',
                                          train_volume_size=5, # 5 GB
                                          output_path='s3://{}/{}/output'.format(bucket_name, 'prefix'),
                                          sagemaker_session=sagemaker.Session())


train_input = s3_input(s3_data='s3://{}/{}/{}'.format(bucket_name, 'train', train_file), content_type='csv')
test_input = s3_input(s3_data='s3://{}/{}/{}'.format(bucket_name, 'test', test_file), content_type='csv')


estimator.fit({'train': train_input, 'validation': test_input})




In [ ]:
# Déployer le modèle
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

In [ ]:
# Prédire les valeurs de l'ensemble de test
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import StringDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = StringDeserializer()

# Serialize the input data without header and index
#payload = X_test.to_csv(header=False, index=False).encode('utf-8')

# Get predictions from the endpoint
predictions = predictor.predict(X_test)

# Convert the prediction response from string to numpy array
predictions_array = np.fromstring(predictions, sep=',')

# Arrondir les prédictions à 0 ou 1
predictions_array = np.round(predictions_array)

# Calculer l'accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions_array)
print(f'Accuracy: {accuracy}')

Accuracy: 0.5865921787709497


In [ ]:
print("Nombre de caractéristiques dans les données d'entraînement : ", X_train.shape[1])
print("Nombre de caractéristiques dans les données d'inférence : ", X_test.shape[1])
print("Les caractéristiques sont dans le même ordre : ", all(X_train.columns == X_test.columns))
print("Il y a des données manquantes dans les données d'inférence : ", X_test.isnull().any().any())
#print(payload)

In [ ]:
predictor.delete_endpoint()
